## Adaptation de namedGame à Jupyter

Vous pouvez utiliser cette feuille pour visualiser un match.

Pour charger vos joueurs, il faut changer leurs noms dans 'classNames' (ligne 13)

In [1]:
''' Sorry no comments :).
'''
import Goban 
import importlib
import time
from io import StringIO
import sys
from IPython.display import display, HTML, clear_output

# Uncomment for saving the successive SVG to PNG
# from cairosvg import svg2png

# Define here the players to load
classNames = ['aliPlayer', 'randomPlayer']

b = Goban.Board()

players = []
player1class = importlib.import_module(classNames[0])
player1 = player1class.myPlayer()
player1.newGame(Goban.Board._BLACK)
players.append(player1)

player2class = importlib.import_module(classNames[1])
player2 = player2class.myPlayer()
player2.newGame(Goban.Board._WHITE)
players.append(player2)

totalTime = [0,0] # total real time for each player
nextplayer = 0
nextplayercolor = Goban.Board._BLACK
nbmoves = 1

outputs = ["",""]
sysstdout= sys.stdout
stringio = StringIO()
wrongmovefrom = 0

print("The game is setup with " + player1.getPlayerName() + " playing BLACKS against " + player2.getPlayerName() + " playing WHITE")
print("Engines loaded. Ready to go.")


The game is setup with Budah playing BLACKS against Random Player playing WHITE
Engines loaded. Ready to go.


In [2]:
coup = 1
while not b.is_game_over():
    print("Referee Board:")
    clear_output(wait=True)
    display(HTML(b.svg()))
    #svg2png(bytestring=b.svg(),write_to='output-'+str(coup).zfill(4)+'.png')
    legals = b.legal_moves() # legal moves are given as internal (flat) coordinates, not A1, A2, ...

    print("Legal Moves: ", [b.move_to_str(m) for m in legals]) # I have to use this wrapper if I want to print them
    nbmoves += 1
    otherplayer = (nextplayer + 1) % 2
    othercolor = Goban.Board.flip(nextplayercolor)
    
    currentTime = time.time()
    #sys.stdout = stringio
    move = players[nextplayer].getPlayerMove() # The move must be given by "A1", ... "J8" string coordinates (not as an internal move)
    #sys.stdout = sysstdout
    playeroutput = stringio.getvalue()
    stringio.truncate(0)
    stringio.seek(0)
    print(("[Player "+str(nextplayer) + "] ").join(playeroutput.splitlines(True)))
    outputs[nextplayer] += playeroutput
    totalTime[nextplayer] += time.time() - currentTime
    print("Player ", nextplayercolor, players[nextplayer].getPlayerName(), "plays: " + move) #changed 

    if not Goban.Board.name_to_flat(move) in legals:
        print(otherplayer, nextplayer, nextplayercolor)
        print("Problem: illegal move")
        wrongmovefrom = nextplayercolor
        break
    b.push(Goban.Board.name_to_flat(move)) # Here I have to internally flatten the move to be able to check it.
    players[otherplayer].playOpponentMove(move)
 
    nextplayer = otherplayer
    nextplayercolor = othercolor
    time.sleep(1)
    coup += 1

print("The game is over")
b.prettyPrint()
result = b.result()
print("Time:", totalTime)
print("GO Score:", b.final_go_score())
print("Winner: ", end="")
if wrongmovefrom > 0:
    if wrongmovefrom == b._WHITE:
        print("BLACK")
    elif wrongmovefrom == b._BLACK:
        print("WHITE")
    else:
        print("ERROR")
elif result == "1-0":
    print("WHITE")
elif result == "0-1":
    print("BLACK")
else:
    print("DEUCE")

# To convert png files to animated gif    
#convert -delay 10 -loop 0 *.png -background white -alpha remove -alpha off output.gif



Legal Moves:  ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'J1', 'A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'J2', 'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'J4', 'A5', 'B5', 'C5', 'D5', 'F5', 'G5', 'H5', 'J5', 'A6', 'B6', 'C6', 'D6', 'F6', 'G6', 'H6', 'J6', 'A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'J7', 'A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'J8', 'A9', 'B9', 'D9', 'F9', 'G9', 'H9', 'J9', 'PASS']
-------------------------------- TURN  4  --------------------
time:  0.017073392868041992


KeyboardInterrupt: 